In [1]:
import pandas as pd

# Read unzipped CSVs

In [2]:
# codes for each file-type and list of years used

filecodes = ['B02001', 'S0101', 'S1401', 'S1501', 'S1601', 'S1901',
             'S1902', 'S1903', 'S2001', 'S2101', 'S2201', 'S2301']

years = [12, 13, 14, 15, 16]

In [3]:
def reader(filecodes, years):
    
    '''
    read all the csv's to their own dataframes
    
    return a list of n*m dataframes... wow
    '''
    
    df_list = []
    
    for year in years:
                
        for filecode in filecodes:
            
            df_list.append(
                pd.read_csv(
                '/home/nate/ds/metis/class_work/projects/raw_data/acs{}/ACS_{}_1YR_{}_with_ann.csv'
                .format(year, year, filecode)))
        
            
    return df_list

In [4]:
# list of all 60 dataframes and check len

df_list = reader(filecodes, years)

len(df_list)

60

# Clean-up Columns

In [5]:
def remake_ids(df_list):
    
    '''
    all dataframes start with col_names in zero-th row and two id cols 
    
    remove these
    '''
    
    no_ids = []

    for df in df_list:

        df.columns = df.iloc[0]

        df = df.drop(0).reset_index()

        df = df.drop(df.columns[0:3], axis=1)
        
        no_ids.append(df)
        
    return no_ids

In [6]:
no_ids = remake_ids(df_list)

In [7]:
len(no_ids)

60

In [8]:
df = no_ids[9]

list(df)

['Geography',
 'Total; Estimate; Civilian population 18 years and over',
 'Total; Margin of Error; Civilian population 18 years and over',
 'Veterans; Estimate; Civilian population 18 years and over',
 'Veterans; Margin of Error; Civilian population 18 years and over',
 'Nonveterans; Estimate; Civilian population 18 years and over',
 'Nonveterans; Margin of Error; Civilian population 18 years and over',
 'Total; Estimate; PERIOD OF SERVICE - Gulf War (9/2001 or later) veterans',
 'Total; Margin of Error; PERIOD OF SERVICE - Gulf War (9/2001 or later) veterans',
 'Veterans; Estimate; PERIOD OF SERVICE - Gulf War (9/2001 or later) veterans',
 'Veterans; Margin of Error; PERIOD OF SERVICE - Gulf War (9/2001 or later) veterans',
 'Nonveterans; Estimate; PERIOD OF SERVICE - Gulf War (9/2001 or later) veterans',
 'Nonveterans; Margin of Error; PERIOD OF SERVICE - Gulf War (9/2001 or later) veterans',
 'Total; Estimate; PERIOD OF SERVICE - Gulf War (8/1990 to 8/2001) veterans',
 'Total; Margi

# Clean-up by Filecodes and Re-name Columns

In [38]:
def file_cleaner(df_list):
    
    '''
    takes the list of df's
    
    9 per year, 5 years, down to 45 from 60
    
    cleans according to filetype, returns list of clean dfs
    
    PRE-MVP: down to 25 dfs, 5 per year, single variable per df
    '''
    
    clean_dfs = []
    
    for i, df in enumerate(df_list):
        
        # for 0 (race)
###### just white ratio for now, come back after MVP
        if i%12==0:
            df = df.drop(df.columns[2::2], axis=1)
            df = df.drop(df.columns[9:], axis=1)
#             df.columns = ['geo', 'total', 'white', 'black', 'american_indian',
#                           'asian', 'native_hawaiian', 'two_plus']
            df = df.drop(df.columns[3:], axis=1)
            x = []
            j=0
            for row in df.iterrows():
                x.append(float(df.iloc[j][2]) / (float(df.iloc[j][1])))
                j+=1
            x = pd.Series(x)   
            df = df[['Geography']]
            df['white_ratio'] = x
            df.columns = ['geography', 'white_ratio']        
            clean_dfs.append(df)
            
        # for 1 (age+sex)
###### just sex for now, come back after MVP
        elif i%12==1:
            df = df[['Geography', 'Male; Estimate; Total population', 'Female; Estimate; Total population']]
#             df = df.drop(df.columns[2::2], axis=1)
#             df = df.drop(df.columns[5::3], axis=1)
#             df = df.drop(df.columns[5::2], axis=1)
#             df = df.drop(df.columns[22:], axis=1)
            x = []
            j=0
            for row in df.iterrows():
                x.append(float(df.iloc[j][2]) / (float(df.iloc[j][1]) + float(df.iloc[j][2])))
                j+=1
            x = pd.Series(x)   
            df = df[['Geography']]
            df['female_ratio'] = x
            df.columns = ['geography', 'female_ratio']
            clean_dfs.append(df)
            
        # for 2 (school enrollment)
###### come back after MVP
        elif i%12==2:
            pass
#             cols = [col for col in df.columns if col[:24] == 'Total; Estimate; Percent']
#             df = pd.concat([df['Geography'], df[cols]], axis=1)
        
        # for 3 (highest education)
        elif i%12==3:
            try:
                df = df[['Geography', "Total; Estimate; Associate's degree", 
                         "Total; Estimate; Bachelor's degree", "Total; Estimate; Graduate or professional degree"]]
                x = []
                j=0
                for row in df.iterrows():
                    x.append((float(df.iloc[j][1]) + float(df.iloc[j][2]) + float(df.iloc[j][3]))/100)
                    j+=1
                x = pd.Series(x)   
                df = df[['Geography']]
                df['college_ratio'] = x
                df.columns = ['geography', 'college_ratio']
                clean_dfs.append(df)
            except:
                df = df[['Geography', 'Total; Estimate; Population 25 years and over',
                         "Total; Estimate; Population 25 years and over - Associate's degree",
                         "Total; Estimate; Population 25 years and over - Bachelor's degree",
                         'Total; Estimate; Population 25 years and over - Graduate or professional degree']]
                x = []
                j=0
                for row in df.iterrows():
                    x.append((float(df.iloc[j][2]) + float(df.iloc[j][3]) + float(df.iloc[j][4]))
                             /float(df.iloc[j][1]))
                    j+=1
                x = pd.Series(x)   
                df = df[['Geography']]
                df['college_ratio'] = x
                df.columns = ['geography', 'college_ratio']
                clean_dfs.append(df)
            
        # for 4 (home lang)
        elif i%12==4:
            if i<40:
                df = df[['Geography', 'Total; Estimate; Speak only English']]
                eng = []
                for i, row in df.iterrows():
                    eng.append(float(df.iloc[i][1])/100)
                df = df[['Geography']]
                df['eng'] = eng
                df.columns = ['geography', 'eng_ratio']
                clean_dfs.append(df)
            else:
                df = df[['Geography', 'Total; Estimate; Population 5 years and over',
                         'Total; Estimate; Speak only English']]
                eng = []
                for i, row in df.iterrows():
                    eng.append(float(df.iloc[i][2])/float(df.iloc[i][1]))
                df = df[['Geography']]
                df['eng'] = eng
                df.columns = ['geography', 'eng_ratio']
                clean_dfs.append(df)
            
        # for 5 (Income)
###### come back after MVP                         
        # info for i%12== 6, 7, 8 was found in i%12==5    
        elif i%12==5:
            df = df.drop(df.columns[2::2], axis=1)
            df = df.drop(df.columns[53:], axis=1)
            df = df.drop(df.columns[1:45], axis=1)
            df = df.drop(df.columns[5:], axis=1)
            df.columns = ['geography', 'household_median', 'family_median', 'married_median', 'nonfamily_median']
            clean_dfs.append(df)
            
        elif i%12==6:
            pass
        elif i%12==7:
            pass
        elif i%12==8:
            pass
            
        # for 9 (veteran status)
####### come back after MVP
        elif i%12==9:
#             pass
            df = df[['Geography', 'Total; Estimate; Civilian population 18 years and over',
                     'Veterans; Estimate; Civilian population 18 years and over']]
            x = []
            m=0
            for row in df.iterrows():
                x.append(float(df.iloc[m][2]) / float(df.iloc[m][1]))
                m+=1
            x = pd.Series(x)   
            df = df[['Geography']]
            df['veteran_ratio'] = x
            df.columns = ['geography', 'veteran_ratio']
            clean_dfs.append(df)
            
        # for 10 (food stamps)
###### come back after MVP
        elif i%12==10:  
            pass
#             try:
#                 df = df[['Geography', 'Total; Estimate; Households',
#                          'Households receiving food stamps; Estimate; Households']]
#             except:
#                 df = df[['Geography', 'Total; Estimate; Households',
#                          'Households receiving food stamps/SNAP; Estimate; Households']]
#             x = []
#             n=0
#             for row in df.iterrows():
#                 x.append(float(df.iloc[n][1]) / float(df.iloc[n][2]))
#                 n+=1
#             x = pd.Series(x)   
#             df = df[['Geography']]
#             df['food_stamps'] = x
#             df.columns = ['geography', 'food_stamps']
            
        # for 11 (employment status)    
        else:
            df = df[['Geography', 'Unemployment rate; Estimate; Population 16 years and over']]
            emp = []
            for i, row in df.iterrows():
                emp.append(float(df.iloc[i][1])/100)
            df = df[['Geography']]
            df['emp'] = emp
            df.columns = ['geography', 'unemployment']
            clean_dfs.append(df)
        
    return clean_dfs

In [39]:
clean_dfs = file_cleaner(no_ids)

# Concat dfs by Year

In [40]:
def years(df_list):
    
    '''
    initialize 5 dfs, one for each year
    
    also add col with year
    '''
    
    list_12 = []
    list_13 = []
    list_14 = []
    list_15 = []
    list_16 = []
    
    num_df = len(df_list)/5
    
    for i, df in enumerate(df_list):
        
        if i < num_df:
            list_12.append(df)
        
        elif i < num_df*2:
            list_13.append(df)
            
        elif i < num_df*3:
            list_14.append(df)
            
        elif i < num_df*4:
            list_15.append(df)
            
        elif i < num_df*5:
            list_16.append(df)
            
    
    df_12 = pd.concat([df for df in list_12], axis=1)
    df_12 = df_12.T.drop_duplicates().T
    df_12['year'] = 12
    
    df_13 = pd.concat([df for df in list_13], axis=1)
    df_13 = df_13.T.drop_duplicates().T
    df_13['year'] = 13
    
    df_14 = pd.concat([df for df in list_14], axis=1)
    df_14 = df_14.T.drop_duplicates().T
    df_14['year'] = 14
    
    df_15 = pd.concat([df for df in list_15], axis=1)
    df_15 = df_15.T.drop_duplicates().T
    df_15['year'] = 15
    
    df_16 = pd.concat([df for df in list_16], axis=1)
    df_16 = df_16.T.drop_duplicates().T
    df_16['year'] = 16
    
            
    return df_12, df_13, df_14, df_15, df_16

In [41]:
df_12, df_13, df_14, df_15, df_16 = years(clean_dfs)

In [42]:
df_12.to_csv('/home/nate/ds/metis/class_work/projects/clean_data/df_12.csv')
df_13.to_csv('/home/nate/ds/metis/class_work/projects/clean_data/df_13.csv')
df_14.to_csv('/home/nate/ds/metis/class_work/projects/clean_data/df_14.csv')
df_15.to_csv('/home/nate/ds/metis/class_work/projects/clean_data/df_15.csv')
df_16.to_csv('/home/nate/ds/metis/class_work/projects/clean_data/df_16.csv')